In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras

from common import *
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v1_1 import CNN_v1_1

Using TensorFlow backend.


In [3]:
train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

In [4]:
cnn = CNN_v1_1((480, 640, 3), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v1/weights/"

In [7]:
model_names = ["cnn_v1_1_1e_1_50",
          "cnn_v1_1_1e_1_75",
          "cnn_v1_1_1e_3_10",
          "cnn_v1_1_1e_3_50",
          "cnn_v1_1_1e_3_100",
          "cnn_v1_1_1e_3_300",
          "cnn_v1_1_1e_3_400",
          "cnn_v1_1_1e_0_150"]

model_names = [prefix + name for name in model_names]

In [8]:
models = [CNN_v1_1((480, 640, 3), split_into_parts=20) for i in range(len(model_names))]

In [9]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=3, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.7940 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 1.6530 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.6962 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.7420 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.7075 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 1.9878 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 1.4905 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 3s 3s/step - loss: 1.9657 - acc: 1.0000


In [10]:
for name, result in results.items():
  diff = best_cop_diff(result, count_test)
  
  print(f"Model {name} behaved:")
  
  print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
  print(diff[2])
  
  print(f"With MSE {mse(diff[2])}")
  print(f"With MAE {mae(diff[2])}", end="\n\n")

Model CNN/CNN_v1/weights/cnn_v1_1_1e_1_50 behaved:
For cut-of-point 0.1 had accuracy 0.0:
[-41 -41 -40 -37 -41 -33 -31 -27 -22 -18 -17 -16 -22 -17 -16 -18 -27 -23
 -23 -24 -24 -24 -28 -29 -23 -24 -20 -23 -18 -18 -23 -29 -29 -23 -22 -27
 -24 -23 -24 -26 -23 -24 -26 -18 -18 -27 -28 -30 -26 -23 -25 -19 -18 -19
 -23 -23 -36 -35 -39 -45 -47 -48 -46 -38 -33 -34 -39 -44 -50 -33 -32 -31
 -31 -32 -28 -27 -28 -28 -28 -27 -29 -28 -26 -23 -24 -25 -21 -25 -25 -22
 -23 -29 -25 -45 -42 -41 -42 -41 -36 -34 -32 -34 -30 -28 -30 -31 -34 -38
 -46 -42 -36 -33 -40 -41 -40 -41 -33 -27 -26 -26 -36 -38 -37 -38 -31 -30
 -29 -26 -30 -29 -32 -34 -30 -36 -44 -33 -32 -26 -31 -29 -29 -32 -37 -42
 -46 -38 -37 -28 -34 -28 -28 -30 -31 -36 -36 -34 -34 -37 -31 -28 -31 -30
 -31 -24 -26 -37 -30 -27 -27 -26 -27 -27 -25 -25 -27 -31 -30 -29 -27 -23
 -27 -28 -30 -30 -29 -33 -36 -37 -24 -23 -33 -29 -36 -35 -30 -27 -30 -30
 -30 -34 -21 -22 -28 -31 -25 -22 -22 -27 -27 -33 -34 -35 -38 -39 -41 -31
 -31 -29 -31 -34 -30 -35 -45 -41 -